In [1]:
%pip install flask requests flask_oauthlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
creds = json.loads(open(os.path.normpath(os.path.expanduser('~/.credentials/dexcom_pfun-app_glucose.json'))).read())
os.environ['DEXCOM_CLIENT_ID'] = creds['client_id']
os.environ['DEXCOM_CLIENT_SECRET'] = creds['client_secret']

In [3]:
import os
from flask import Flask, redirect, url_for, session, request
from flask_oauthlib.client import OAuth

app = Flask(__name__)
app.secret_key = 'random_secret_key'  # Change this to a random secret key
oauth = OAuth(app)

dexcom = oauth.remote_app(
    'dexcom',
    consumer_key=os.getenv('DEXCOM_CLIENT_ID'),
    consumer_secret=os.getenv('DEXCOM_CLIENT_SECRET'),
    request_token_params={
        'redirect_uri': 'http://localhost:5000/login/authorized'
    },
    base_url='https://api.dexcom.com/v2/',
    request_token_url=None,
    access_token_method='POST',
    access_token_url='https://api.dexcom.com/v2/oauth2/token',
    authorize_url='https://api.dexcom.com/v2/oauth2/login'
)

@app.route('/')
def index():
    return 'Welcome to Dexcom Wrapper! <a href="/login">Login with Dexcom</a>'

@app.route('/login')
def login():
    return dexcom.authorize(callback=url_for('authorized', _external=True))

@app.route('/login/authorized')
def authorized():
    response = dexcom.authorized_response()
    if response is None or response.get('access_token') is None:
        return 'Access denied: reason={} error={}'.format(
            request.args['error_reason'],
            request.args['error_description']
        )

    session['dexcom_token'] = (response.get('access_token'), '')
    me = dexcom.get('users/self')
    return 'Logged in as: {}'.format(me.data)

@dexcom.tokengetter
def get_dexcom_oauth_token():
    return session.get('dexcom_token')

if __name__ == '__main__':
    app.run(debug=True, port=5000)


ImportError: cannot import name 'url_quote' from 'werkzeug.urls' (d:\RobbieDocuments\Data\.venv\Lib\site-packages\werkzeug\urls.py)